In [66]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.aqua.components.optimizers import GSLS, L_BFGS_B, SLSQP, TNC, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES
import math
import random
import numpy as np
from numpy import pi
import time
import itertools
import pickle

## Auxiliary functions

In [67]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [68]:
def prepare_initial_state(theta, phase):
    """ Prepare initial state """
    return (0, np.sqrt(theta),
        np.sqrt(1-theta), 0)

In [69]:
def get_measured_value_from_counts(counts_dict: dict) -> str:
    """ converts the dictionary counts measured to the
        value measured in string format
    """
    if len(list(counts_dict.keys())) != 1:
        raise ValueError('Circuit execution shots MUST be 1')
    return list(counts_dict.keys())[0]

In [70]:
def convert_counts_to_final_result(counts) -> int:
    """ Convert the execution result to the final measured value: 0 or 1 """
    if counts == "00":
        return 1
    if counts == "10":
        return 0
    else:
        return random.choice([0,1])
    return 1

In [71]:
def set_random_lambda(lambda0, lambda1):
    """ return a random choice from lambda0 and lambda1 with the correspondent key name """
    lambda_value = random.choice([lambda0, lambda1])
    if lambda_value == lambda0:
        return lambda_value, 0
    return lambda_value, 1

In [72]:
def compute_damping_channel(theta, phase, one_eta, phi_rx, phi_ry, backend) -> int:
    """ one-time execution of the amplitude damping circuit using the passed parameters 
        Returns: the execution measured result: either 0 or 1
    """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = prepare_initial_state(theta, 0)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * one_eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.rx(phi_rx, qreg_q[1])
    circuit.ry(phi_ry, qreg_q[1])
    circuit.barrier() 
    circuit.measure([0, 1], creg_c)

    counts2 = execute(circuit, backend, shots=1).result().get_counts(circuit)
    counts = get_measured_value_from_counts(counts2)
    
    return convert_counts_to_final_result(counts)

In [73]:
def compute_random_damping_channel(configuration, backend):
    """ run a one-time execution on the given backend device randomly selectiong the lambda to be used and return the measured result """
    lambda_value, lambda_used = set_random_lambda(configuration['lambda0'], configuration['lambda1'])
    result = compute_damping_channel(configuration['theta'], 0, lambda_value, configuration['phi_rx'], configuration['phi_ry'], backend)

    return {
        "result": result,
        "lambda_used": lambda_used
    }

In [74]:
def guess_lambda_used(real_measured_result):
    """ Decides which lambda was used on the real execution.
        It is a silly guess.
        It returns the same lambda used as the measured result
    """
    return real_measured_result

In [75]:
def check_value(real_lambda, guess_lambda): 
    if real_lambda == guess_lambda: return 1
    return 0

In [76]:
def play_and_guess_one_case(configuration, backend):
    """ Execute a real execution with a random lambda from the two passed, 
        guess which one was used on the exection and
        check the result.
        Returns 1 on success (it was a correct guess) or 0 on fail (it was an incorrect guess) 
    """
    real_result = compute_random_damping_channel(configuration, backend)
    guess_lambda = guess_lambda_used(real_result['result'])
    return check_value(real_result['lambda_used'], guess_lambda)

In [77]:
def get_combinations_two_etas_without_repeats(input_list_eta):
    """ from a given list of attenuation angles (etas) create a 
        list of all combinatorial pairs of possible etas
        without repeats (order does not matter).
        For us it is the same testing first eta 0.1 and second eta 0.2 
        than first eta 0.2 and second eta 0.1
    """
    list_eta = input_list_eta
     # when there is only one element, we add the same element
    if len(list_eta) == 1:
        list_eta.append(list_eta[0])
    # get combinations of two etas without repeats
    return list(itertools.combinations(list_eta, 2))

## Creating our cost function to be minimized (minimizing the error)

In [78]:
def play_one_configuration(configuration, backend, plays=1000):
    """ Play the play_and_guess game for the number of plays for the configuration of input parameters.
        Returns the average success probability for all the plays with the given configuration. 
    """
    success_counts = 0
    for play in range(plays):
        success_counts += play_and_guess_one_case(configuration, backend)
    return success_counts/plays

In [79]:
def cost_function(params):
    """ Computes the cost of running a specific configuration.
        Cost is computed as 1 (perfect probability) - given probability [0,1]
    """
    backend = backend=Aer.get_backend('qasm_simulator')

    configuration = {
        "theta": params[0],
        "phi_rx": params[1],
        "phi_ry": params[2],
        "lambda0": GLOBAL_LAMBDA0,
        "lambda1": GLOBAL_LAMBDA1
    }
    return -play_one_configuration(configuration, backend, plays=400)

## Parameter Optimization for the best configuration

In [80]:
def compute_best_configuration( optimizer_algorithms=['CRS'], 
                                list_optimizer_iterations=[1000], 
                                list_step_size=[pi/10], 
                                list_iterations_per_algorithm=[1],
                                initial_params = [0, 0, 0]):
    """ Find out the best configuration with a global pair of lambdas (channels) trying out a list of specified optimizat algorithm """
    # Create the initial parameters and set the bounds
    #initial_params = [0, 0, 0, 0]
    variable_bounds = [
                        (0, 1), # Prob. state |10> usage of ancilla
                        (0, 2*pi), # rx
                        (0, 2*pi)  # ry
                        ]
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""

    for optimizer_algorithm, optimizer_iterations, step_size, iterations in zip(optimizer_algorithms, 
                                                                    list_optimizer_iterations, 
                                                                    list_step_size, 
                                                                    list_iterations_per_algorithm):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'GSLS':
            # optimizer = GSLS()
            optimizer = GSLS(maxiter=optimizer_iterations, max_eval=optimizer_iterations, initial_step_size=step_size)
        if optimizer_algorithm == 'L_BFGS_B':
            # optimizer = L_BFGS_B()
            optimizer = L_BFGS_B(maxfun=optimizer_iterations, maxiter=optimizer_iterations, epsilon=step_size)
        if optimizer_algorithm == 'SLSQP':
            # optimizer = SLSQP()
            optimizer = SLSQP(maxiter=optimizer_iterations,  eps=step_size)
        if optimizer_algorithm == 'TNC':
            # optimizer = TNC()
            optimizer = TNC(maxiter=optimizer_iterations, eps=step_size)
        if optimizer_algorithm == 'BOBYQA':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'IMFIL':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'SNOBFIT':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=optimizer_iterations)

        
        for i in range(iterations):
            ret = optimizer.optimize(num_vars=len(initial_params), 
                                    objective_function=cost_function, 
                                    variable_bounds=variable_bounds,
                                    initial_point=initial_params)
            print("Best Average Probability:", 1 - ret[1])
            if (1-ret[1]) > best_probability:
                best_probability = 1-ret[1]
                best_configuration = ret[0]
                best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Parameters Found: " + u"\u03B8" + " = " + str(int(math.degrees(best_configuration[0]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "rx = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "ry = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" +
                            ", " + u"\u03B7" + u"\u2080" + " = " + str(GLOBAL_LAMBDA0) + 
                            ", " + u"\u03B7" + u"\u2081" + " = " + str(GLOBAL_LAMBDA1))

    return {
        "best_optimizer_algorithm": best_optimizer_algorithm,
        "best_probability": best_probability,
        "best_configuration": best_configuration
    }


## Find out the best configuration ($\theta, phase, rx, ry$) and success probability for each pair of $\lambda$ attenuation levels from 0 to 1

In [81]:
""" Load results from a file """
loaded_results = load_results_from_file(name='20210405a_C2_AX')

In [ ]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['TNC']
list_optimizer_iterations=[1]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)
eta_pair_ini = 0
eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")
i=0
for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    GLOBAL_LAMBDA0=eta_pair[0]
    GLOBAL_LAMBDA1=eta_pair[1]
    initial_params=[loaded_results['configurations'][i][0], loaded_results['configurations'][i][1], loaded_results['configurations'][i][2]]
    result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm,
                                        initial_params=initial_params)
    probabilities.append(result['best_probability'])
    configurations.append(result['best_configuration'])
    best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))
    i=i+1
end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.4875
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.4875
Parameters Found: θ = 0°, ϕrx = 238°, ϕry = 354°, η₀ = 0.0, η₁ = 0.07853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.48
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.48
Parameters Found: θ = 47°, ϕrx = 140°, ϕry = 220°, η₀ = 0.0, η₁ = 0.15707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.505
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.505
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.0, η₁ = 0.23561944901923448
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.5375
Final Be

Best Average Probability: 1.7225000000000001
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.7225000000000001
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 9°, η₀ = 0.07853981633974483, η₁ = 0.7068583470577035
total minutes taken this pair of etas:  1
total minutes taken so far:  10
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.73
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.73
Parameters Found: θ = 2°, ϕrx = 0°, ϕry = 0°, η₀ = 0.07853981633974483, η₁ = 0.7853981633974483
total minutes taken this pair of etas:  1
total minutes taken so far:  11
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.7774999999999999
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.7774999999999999
Parameters Found: θ = 1°, ϕrx = 0°, ϕry = 0°, η₀ = 0.07853981633974483, η₁ = 0.8639379797371931
total minutes taken this pair of etas:  1
total minutes taken so far:  12
Analyzing Optimizer Algorithm:  TNC
Best Averag

Best Average Probability: 1.9725000000000001
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.9725000000000001
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.335176877775662
total minutes taken this pair of etas:  0
total minutes taken so far:  20
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.9825
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.9825
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.413716694115407
total minutes taken this pair of etas:  0
total minutes taken so far:  21
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.9925000000000002
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.9925000000000002
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.4922565104551517
total minutes taken this pair of etas:  0
total minutes taken so far:  21
Analyzing Optimizer Algorithm:  TNC
Best Aver

Best Average Probability: 1.6425
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.6425
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 0.7068583470577035
total minutes taken this pair of etas:  0
total minutes taken so far:  30
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.7349999999999999
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.7349999999999999
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 0.7853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  30
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.7325
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.7325
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 0.8639379797371931
total minutes taken this pair of etas:  0
total minutes taken so far:  30
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.77
Fin

Best Average Probability: 1.905
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.905
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.39269908169872414, η₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  40
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.575
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.575
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 360°, η₀ = 0.47123889803846897, η₁ = 0.5497787143782138
total minutes taken this pair of etas:  0
total minutes taken so far:  41
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.565
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.565
Parameters Found: θ = 3°, ϕrx = 353°, ϕry = 18°, η₀ = 0.47123889803846897, η₁ = 0.6283185307179586
total minutes taken this pair of etas:  1
total minutes taken so far:  41
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.645
Final Best Optimizer Alg

Best Average Probability: 1.85
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.85
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.5497787143782138, η₁ = 1.4922565104551517
total minutes taken this pair of etas:  0
total minutes taken so far:  50
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.8625
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.8625
Parameters Found: θ = 3°, ϕrx = 180°, ϕry = 166°, η₀ = 0.5497787143782138, η₁ = 1.5707963267948966
total minutes taken this pair of etas:  1
total minutes taken so far:  51
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.5125
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.5125
Parameters Found: θ = 9°, ϕrx = 220°, ϕry = 175°, η₀ = 0.6283185307179586, η₁ = 0.7068583470577035
total minutes taken this pair of etas:  0
total minutes taken so far:  51
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.595
Final Best Optimizer A

Best Average Probability: 1.55
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.55
Parameters Found: θ = 9°, ϕrx = 8°, ϕry = 360°, η₀ = 0.7853981633974483, η₁ = 0.9424777960769379
total minutes taken this pair of etas:  0
total minutes taken so far:  59
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.615
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.615
Parameters Found: θ = 8°, ϕrx = 180°, ϕry = 184°, η₀ = 0.7853981633974483, η₁ = 1.0210176124166828
total minutes taken this pair of etas:  0
total minutes taken so far:  59
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.62
Final Best Optimizer Algorithm:  TNC
Final Best Average Probability: 1.62
Parameters Found: θ = 1°, ϕrx = 179°, ϕry = 193°, η₀ = 0.7853981633974483, η₁ = 1.0995574287564276
total minutes taken this pair of etas:  0
total minutes taken so far:  60
Analyzing Optimizer Algorithm:  TNC
Best Average Probability: 1.6825
Final Best Optimizer Algo

In [63]:
""" Save results to disk """
save_results_to_disk(result, name='20210405a_F2b_C2_AX_OL-TNC')